# Projeto Integração I

# Importação de Bibliotecas

In [16]:
import numpy as np
import pandas as pd

# Importação do Data Frame

Identificamos que os dados a serem lidos estão em um arquivo .csv e usamos o método read_csv do pandas para passar os dados em um formato de DataFrame

In [17]:
data_location = "../Data/Real_State_Properati.csv"

df_prop = pd.read_csv(data_location, sep = '\t')


# Encontramos os tipos de cada coluna
Identificamos os tipos de cada coluna e printamos alguns dados para melhor entendimento dos dados

In [18]:
print(df_prop.dtypes)
df_prop.head()

property_type           object
place_name              object
country_name            object
state_name              object
lat                    float64
lon                    float64
price                  float64
currency                object
surface_total_in_m2    float64
dtype: object


,property_type,place_name,country_name,state_name,lat,lon,price,currency,surface_total_in_m2
0,PH,Mataderos,Argentina,Capital Federal,-34.661824,-58.508839,62000.0,USD,55.0
1,apartment,Mataderos,Argentina,Capital Federal,-34.652262,-58.522982,72000.0,USD,55.0
2,apartment,Belgrano,Argentina,Capital Federal,-34.559873,-58.443362,138000.0,USD,45.0
3,apartment,Belgrano,Argentina,Capital Federal,-34.559873,-58.443362,195000.0,USD,65.0
4,PH,Mataderos,Argentina,Capital Federal,-34.652356,-58.501624,239000.0,USD,140.0


# Modificando nome da coluna surface_total_in_m2 para total_m2
Facilitará quando precisarmos analisar e usar esse campo

In [19]:
df_prop.rename(columns={'surface_total_in_m2': 'total_m2'}, inplace = True)
df_prop.columns

Index(['property_type', 'place_name', 'country_name', 'state_name', 'lat',
       'lon', 'price', 'currency', 'total_m2'],
      dtype='object')

# Verificação de valores ausentes
Verificamos os valores ausentes em cada coluna. Conclusão: Não existem valores ausentes em nenhuma das colunas

In [20]:
data_null = df_prop.apply(lambda x : x.isnull().sum())
data_null

property_type    0
place_name       0
country_name     0
state_name       0
lat              0
lon              0
price            0
currency         0
total_m2         0
dtype: int64

# Verificação de valores menores ou iguais a zero nas colunas price e total_m2
Vamos verificar se existem valores menores ou iguais a zero nas colunas price e surface_total_in_m2. Sobre a coluna price não encontramos nenhum registro com valor menor ou igual a zero, mas sobre a coluna total_m2 encontramos 6 registros.

In [21]:
data_menor_igual_zero = pd.DataFrame(df_prop, columns=['price','total_m2'])
data_menor_igual_zero = data_menor_igual_zero.apply(lambda x : x<=0).sum()
data_menor_igual_zero

price       0
total_m2    6
dtype: int64

# Verificando quais registros de total_m2 estão com valor menor ou igual a zero
Temos 6 registros com valor em total_m2 menor ou igual a zero. Nas próximas linhas veremos qual ação tomar diante desse fato.

In [22]:
mask_m2_menor_igual_zero = df_prop['total_m2']<=0
data_m2_menor_igual_zero = df_prop.loc[mask_m2_menor_igual_zero]
data_m2_menor_igual_zero

,property_type,place_name,country_name,state_name,lat,lon,price,currency,total_m2
2486,apartment,Palermo Soho,Argentina,Capital Federal,-34.589974,-58.425614,780000.0,USD,0.0
4872,apartment,Recoleta,Argentina,Capital Federal,-34.588818,-58.390648,320000.0,USD,0.0
5454,apartment,Caballito,Argentina,Capital Federal,-34.617348,-58.449404,68500.0,USD,0.0
6318,apartment,Palermo Hollywood,Argentina,Capital Federal,-34.582558,-58.432585,175000.0,USD,0.0
7170,apartment,Barracas,Argentina,Capital Federal,-34.641131,-58.369122,135000.0,USD,0.0
15888,apartment,Barrio Norte,Argentina,Capital Federal,-34.598727,-58.395851,112000.0,USD,0.0


# Vamos preencher o total_m2 usando a média por property_type e place_name
Primeiro iremos criar um novo dataframe apenas com os registros cujo total_m2 seja maior que zero. Depois disso aplicaremos a média de total_m2 por property_type e place_name e preencheremos os valores faltantes na coluna total_m2 com a média respectiva por property_name e place_name

In [23]:
mask_m2_maior_zero = df_prop['total_m2']>0
data_m2_maior_zero = df_prop.loc[mask_m2_maior_zero]
data_m2_maior_zero = pd.DataFrame(data_m2_maior_zero, columns=['property_type','place_name','total_m2'])
data_m2_maior_zero

,property_type,place_name,total_m2
0,PH,Mataderos,55.0
1,apartment,Mataderos,55.0
2,apartment,Belgrano,45.0
3,apartment,Belgrano,65.0
4,PH,Mataderos,140.0
...,...,...,...
17218,apartment,Barrio Norte,91.0
17219,apartment,Recoleta,44.0
17220,apartment,Belgrano,157.0
17221,apartment,Belgrano,157.0


# Agrupando as médias de total_m2 por property_type e place_name

In [24]:
data_media_m2_pp = data_m2_maior_zero.groupby(['property_type','place_name']).mean().reset_index()
data_media_m2_pp

,property_type,place_name,total_m2
0,PH,Abasto,116.100000
1,PH,Agronomía,123.333333
2,PH,Almagro,149.529412
3,PH,Balvanera,149.153846
4,PH,Barracas,161.142857
...,...,...,...
224,store,Villa Ortuzar,386.500000
225,store,Villa Real,520.000000
226,store,Villa Soldati,9505.500000
227,store,Villa Urquiza,125.500000


# Atribuindo as médias de total_m2 agrupadas nas respectivas linhas respeitando property_type e place_name
Usando laço for para atualizar a média do total_m2 por property_type e place_name e depois realizando um check para confirmar que não exista nenhum registro com o total_m2 menor ou igual a zero.

In [25]:
for i in df_prop.index:
    if df_prop.loc[i, 'total_m2']<=0:
        property_type = df_prop.loc[i, 'property_type']
        place_name    = df_prop.loc[i, 'place_name']
        total_m2      = float(data_media_m2_pp[(data_media_m2_pp['property_type']==property_type) & (data_media_m2_pp['place_name']==place_name)]['total_m2'])
        df_prop.loc[i, 'total_m2'] = total_m2

In [26]:
mask_m2_menor_igual_zero_check = df_prop['total_m2']<=0
data_m2_menor_igual_zero_check = df_prop.loc[mask_m2_menor_igual_zero_check]
data_m2_menor_igual_zero_check

,property_type,place_name,country_name,state_name,lat,lon,price,currency,total_m2


In [27]:
df_prop.value_counts('property_type')

property_type
apartment    14428
PH            1142
house          888
store          765
dtype: int64

In [28]:
df_prop.value_counts('currency')

currency
USD    16700
ARS      523
dtype: int64

In [59]:
df_prop.value_counts('place_name')

place_name
Palermo            1502
Belgrano           1496
Caballito          1389
Recoleta           1001
Flores              893
                   ... 
Parque Chas          15
Palermo Viejo        10
Villa Soldati         7
Villa Riachuelo       3
Catalinas             1
Length: 62, dtype: int64

In [30]:
'''criando coluna para unificação de preços uma vez que temos preços em Pesos e Dólares. Cotação do Dólar em 117.08 11/05/2022'''
df_prop['price_usd'] = df_prop.apply(lambda x: x.price/1 if x.currency=='USD' else x.price/117.08, axis=1)
df_prop.head(10)

,property_type,place_name,country_name,state_name,lat,lon,price,currency,total_m2,price_usd
0,PH,Mataderos,Argentina,Capital Federal,-34.661824,-58.508839,62000.0,USD,55.0,62000.0
1,apartment,Mataderos,Argentina,Capital Federal,-34.652262,-58.522982,72000.0,USD,55.0,72000.0
2,apartment,Belgrano,Argentina,Capital Federal,-34.559873,-58.443362,138000.0,USD,45.0,138000.0
3,apartment,Belgrano,Argentina,Capital Federal,-34.559873,-58.443362,195000.0,USD,65.0,195000.0
4,PH,Mataderos,Argentina,Capital Federal,-34.652356,-58.501624,239000.0,USD,140.0,239000.0
5,apartment,Palermo,Argentina,Capital Federal,-34.580504,-58.405874,350000.0,USD,104.0,350000.0
6,apartment,Palermo,Argentina,Capital Federal,-34.590926,-58.411665,270500.0,USD,118.0,270500.0
7,apartment,Flores,Argentina,Capital Federal,-34.635118,-58.473964,75000.0,USD,43.0,75000.0
8,apartment,Boedo,Argentina,Capital Federal,-34.621769,-58.422108,145000.0,USD,80.0,145000.0
9,apartment,Las Cañitas,Argentina,Capital Federal,-34.571150,-58.423297,480000.0,USD,220.0,480000.0


In [33]:
# substituindo todas as moedas pesos pos dolar em currency
df_prop = df_prop.replace({'currency': 'ARS'}, 'USD')
df_prop.value_counts('currency')

currency
USD    17223
dtype: int64

In [34]:
# calcular o preço por m2 na coluna price_m2 levando em consideração que não podemos dividir por 0:
df_prop['price_m2'] = df_prop.apply(lambda x: x['price_usd'] / x['total_m2'], axis=1)
df_prop.head(5)

,property_type,place_name,country_name,state_name,lat,lon,price,currency,total_m2,price_usd,price_m2
0,PH,Mataderos,Argentina,Capital Federal,-34.661824,-58.508839,62000.0,USD,55.0,62000.0,1127.272727
1,apartment,Mataderos,Argentina,Capital Federal,-34.652262,-58.522982,72000.0,USD,55.0,72000.0,1309.090909
2,apartment,Belgrano,Argentina,Capital Federal,-34.559873,-58.443362,138000.0,USD,45.0,138000.0,3066.666667
3,apartment,Belgrano,Argentina,Capital Federal,-34.559873,-58.443362,195000.0,USD,65.0,195000.0,3000.000000
4,PH,Mataderos,Argentina,Capital Federal,-34.652356,-58.501624,239000.0,USD,140.0,239000.0,1707.142857


In [36]:
# verificando se há valores com 0 na coluna de preço por metro quadrado
mask_price_m2_menor_igual_zero = df_prop['price_m2']<= 0
data_price_m2_menor_igual_zero = df_prop.loc[mask_price_m2_menor_igual_zero]
data_price_m2_menor_igual_zero

,property_type,place_name,country_name,state_name,lat,lon,price,currency,total_m2,price_usd,price_m2


# Identificando place_name com menor número de imóveis

In [48]:
df_place_property = pd.DataFrame(df_prop, columns=['property_type','place_name'])
df_place_property['count']=1
df_place_property = df_place_property.groupby(['property_type','place_name']).count().reset_index()
df_place_property

,property_type,place_name,count
0,PH,Abasto,10
1,PH,Agronomía,3
2,PH,Almagro,51
3,PH,Balvanera,26
4,PH,Barracas,35
...,...,...,...
224,store,Villa Ortuzar,4
225,store,Villa Real,1
226,store,Villa Soldati,2
227,store,Villa Urquiza,10


In [79]:
mask_place_property_menor_10 = df_place_property['count']<10
data_place_property_menor_10 = df_place_property.loc[mask_place_property_menor_10]
data_place_property_menor_10['count'].sum()

331

In [80]:
data_place_property_menor_10['count'].sum() / df_prop.shape[0] * 100

1.9218486907042909

In [37]:
df_prop['price_m2'].sort_values(ascending=True)

4308          6.084120
2556         10.544652
9104         10.566394
8744         13.336210
440          13.488512
             ...      
9293      39899.594595
9292      40720.405797
15399     43640.000000
15398     44900.000000
3901     206333.333333
Name: price_m2, Length: 17223, dtype: float64

In [38]:
df_prop.loc[4308]

property_type                   store
place_name       Centro / Microcentro
country_name                Argentina
state_name            Capital Federal
lat                        -34.602029
lon                         -58.37669
price                        260000.0
currency                          USD
total_m2                        365.0
price_usd                 2220.703792
price_m2                      6.08412
Name: 4308, dtype: object

In [23]:
# calcular a média de price_m2 por place_name e vamos imputar os valores ausentes usando essa média condicional.
fill_values = df_prop.groupby('place_name').price_m2.mean()
fill_values.index

Index(['Abasto', 'Agronomía', 'Almagro', 'Balvanera', 'Barracas',
       'Barrio Norte', 'Belgrano', 'Boca', 'Boedo', 'Caballito',
       'Capital Federal', 'Catalinas', 'Centro / Microcentro', 'Chacarita',
       'Coghlan', 'Colegiales', 'Congreso', 'Constitución', 'Flores',
       'Floresta', 'Las Cañitas', 'Liniers', 'Mataderos', 'Monserrat',
       'Monte Castro', 'Nuñez', 'Once', 'Palermo', 'Palermo Chico',
       'Palermo Hollywood', 'Palermo Soho', 'Palermo Viejo',
       'Parque Avellaneda', 'Parque Centenario', 'Parque Chacabuco',
       'Parque Chas', 'Parque Patricios', 'Paternal', 'Pompeya',
       'Puerto Madero', 'Recoleta', 'Retiro', 'Saavedra', 'San Cristobal',
       'San Nicolás', 'San Telmo', 'Tribunales', 'Velez Sarsfield',
       'Versalles', 'Villa Crespo', 'Villa Devoto', 'Villa General Mitre',
       'Villa Lugano', 'Villa Luro', 'Villa Ortuzar', 'Villa Pueyrredón',
       'Villa Real', 'Villa Riachuelo', 'Villa Santa Rita', 'Villa Soldati',
       'Villa Urquiz

In [24]:
# Construímos um DataFrame com duas colunas (place_name e price_m2) a partir da instância Series calculada na etapa anterior:
fill_values_df = pd.DataFrame(fill_values)
fill_values_df.reset_index(inplace=True)
fill_values_df

,place_name,price_m2
0,Abasto,1998.555785
1,Agronomía,2088.124134
2,Almagro,2541.956139
3,Balvanera,1964.499761
4,Barracas,1990.105681
...,...,...
57,Villa Riachuelo,1110.690766
58,Villa Santa Rita,1880.717836
59,Villa Soldati,743.666758
60,Villa Urquiza,2322.547354


In [25]:
# Combinamos o DataFrame original com os valores médios de CouncilArea:

df_prop = df_prop.merge(fill_values_df, on = "place_name", suffixes = ("", "_place_mean"), how = 'left')
df_prop.head(3)

,property_type,place_name,country_name,state_name,lat,lon,price,currency,surface_total_in_m2,price_usd,price_m2,price_m2_place_mean
0,PH,Mataderos,Argentina,Capital Federal,-34.661824,-58.508839,62000.0,USD,55.0,62000.0,1127.272727,1508.225256
1,apartment,Mataderos,Argentina,Capital Federal,-34.652262,-58.522982,72000.0,USD,55.0,72000.0,1309.090909,1508.225256
2,apartment,Belgrano,Argentina,Capital Federal,-34.559873,-58.443362,138000.0,USD,45.0,138000.0,3066.666667,2913.987724


In [26]:
df_prop[['place_name','price_m2','price_m2_place_mean']].sort_values('price_m2')

,place_name,price_m2,price_m2_place_mean
4308,Centro / Microcentro,6.084120,2568.828236
2556,Floresta,10.544652,1713.028114
9104,Flores,10.566394,2060.479653
8744,Parque Chacabuco,13.336210,1664.747327
440,Villa Crespo,13.488512,2400.647125
...,...,...,...
9293,Flores,39899.594595,2060.479653
9292,Flores,40720.405797,2060.479653
15399,Palermo,43640.000000,3287.480088
15398,Palermo,44900.000000,3287.480088


# MEDIA DE PREÇO PELO TIPO DE PROPRIEDADE

In [29]:
# calcular a média de price_m2 agrupando por place_name e property_type e vamos imputar os valores ausentes usando essa média condicional.
fill_values = df_prop.groupby(['place_name','property_type']).price_m2.mean()
fill_values

place_name        property_type
Abasto            PH               1555.190277
                  apartment        2118.547612
                  house            1420.192308
                  store            2110.341975
Agronomía         PH               1566.441441
                                      ...     
Villa Urquiza     store            2354.089922
Villa del Parque  PH               1680.539116
                  apartment        2120.309296
                  house            1421.420845
                  store            1524.630662
Name: price_m2, Length: 229, dtype: float64

In [30]:
# Construído um DataFrame com duas colunas (place_name e price_m2) a partir da instância Series calculada na etapa anterior:
fill_values_df = pd.DataFrame(fill_values)
fill_values_df.reset_index(inplace=True)
fill_values_df

,place_name,property_type,price_m2
0,Abasto,PH,1555.190277
1,Abasto,apartment,2118.547612
2,Abasto,house,1420.192308
3,Abasto,store,2110.341975
4,Agronomía,PH,1566.441441
...,...,...,...
224,Villa Urquiza,store,2354.089922
225,Villa del Parque,PH,1680.539116
226,Villa del Parque,apartment,2120.309296
227,Villa del Parque,house,1421.420845


In [31]:
# Combinamos o DataFrame original com os valores médios em fill_values, sendo criada duas colunas novas para com sufixo pl_type_mean
df_prop = df_prop.merge(fill_values_df, on = "place_name", suffixes = ("", "_pl_type_mean"), how = 'left')


In [36]:
df_prop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67489 entries, 0 to 67488
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   property_type               67489 non-null  object 
 1   place_name                  67489 non-null  object 
 2   country_name                67489 non-null  object 
 3   state_name                  67489 non-null  object 
 4   lat                         67489 non-null  float64
 5   lon                         67489 non-null  float64
 6   price                       67489 non-null  float64
 7   currency                    67489 non-null  object 
 8   surface_total_in_m2         67489 non-null  float64
 9   price_usd                   67489 non-null  float64
 10  price_m2                    67489 non-null  float64
 11  price_m2_place_mean         67489 non-null  float64
 12  property_type_pl_type_mean  67489 non-null  object 
 13  price_m2_pl_type_mean       674

In [37]:
# visualizando o agrupamento com a média de preço por metro quadrado das propriedade por bairro e também por tipo de propriedade
# descrição colunas:
    # price_m2: preço por metro quadrado
    # price_m2_place_mean: média de preço por metro quadrado agrupado por bairro
    # price_m2_pl_type_mean: média de preço por metro quadrado agrupado por bairro e tipo de propriedade
df_prop[['place_name','price_m2','price_m2_place_mean','property_type_pl_type_mean','price_m2_pl_type_mean']].sort_values('price_m2')

,place_name,price_m2,price_m2_place_mean,property_type_pl_type_mean,price_m2_pl_type_mean
16782,Centro / Microcentro,6.084120,2568.828236,store,2783.893099
16780,Centro / Microcentro,6.084120,2568.828236,apartment,2508.053257
16781,Centro / Microcentro,6.084120,2568.828236,house,1366.666667
9951,Floresta,10.544652,1713.028114,store,1820.089338
9950,Floresta,10.544652,1713.028114,house,1209.959077
...,...,...,...,...,...
60313,Palermo,44900.000000,3287.480088,store,3928.589829
15195,Boedo,206333.333333,7666.694964,house,4313.014377
15196,Boedo,206333.333333,7666.694964,store,1450.128922
15193,Boedo,206333.333333,7666.694964,PH,1319.863508
